In [1]:
import sys
sys.path.append('..')
import gdspy
import numpy as np
from importlib import reload
from copy import deepcopy
from QCreator import elements
from QCreator import general_sample_creator as creator
from QCreator import meshing
from QCreator.auxiliary_functions import *

tl_core = 20.
tl_gap = 12.
tl_ground = 10.


# resonator_core = 7
# resonator_gap = 4
# resonator_ground = 16
resonator_core = 8
resonator_gap = 7
resonator_ground = 16

pad_offset = 800
pad_element_offset = 1000
qubit_position_offset = 800

coupler_start = pad_offset + pad_element_offset
coupler_delta = 500
coupler_length = 450
num_couplers = 1

In [2]:
reload(gdspy)
reload(creator)
reload(elements)


layers_configuration = {
    'total':0,
    'restricted area':10,
    'for removing':100,
    'JJs':1,
    'air bridges':2,
    'air bridge pads':3,
    'vertical gridlines':15,
    'horizontal gridlines':16,
    'inverted':17
}

sample = creator.Sample('1Q_test',layers_configuration)

#specify sample vertical and horizontal lengths
sample.chip_geometry.sample_vertical_size=4.7e3
sample.chip_geometry.sample_horizontal_size=4.7e3
# num_qubits_line = 3

central_line_y = sample.chip_geometry.sample_vertical_size/2

# ChichkovAB = elements.AirBridge(width=15, 
#                                 length=60, 
#                                 padsize=30, 
#                                 min_spacing = 30,
#                                 layer_configuration=sample.layer_configuration)

chip_edge_ground = elements.ChipEdgeGround(sample.chip_geometry, sample.layer_configuration, sample.pads)
sample.add(chip_edge_ground)

In [3]:
# 1. Create contact pads:
pads_left = []
pads_right = []
for pad_side_id in range(1):
    pad = elements.Pad('pad-left-'+str(pad_side_id), 
                       (pad_offset,  sample.chip_geometry.sample_vertical_size/2), np.pi, tl_core, 
                       tl_gap, tl_ground, 
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                       **elements.default_pad_geometry())
    pads_left.append(pad)
    sample.add(pad)
    pad = elements.Pad('pad-right-'+str(pad_side_id), 
                       (sample.chip_geometry.sample_horizontal_size-pad_offset,  
                        sample.chip_geometry.sample_vertical_size/2), 0, tl_core, 
                       tl_gap, tl_ground, 
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                       **elements.default_pad_geometry())
    pads_right.append(pad)
    sample.add(pad)
    
pads_top = []
pads_bottom = []
for pad_side_id in range(3):
    pad = elements.Pad('pad-bottom-'+str(pad_side_id), 
                       (sample.chip_geometry.sample_horizontal_size/4*(pad_side_id+1), pad_offset), 
                       -np.pi/2, tl_core, tl_gap, tl_ground, 
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                       **elements.default_pad_geometry())
    pads_bottom.append(pad)
    sample.add(pad)
    pad = elements.Pad('pad-top-'+str(pad_side_id), 
                       (sample.chip_geometry.sample_horizontal_size/4*(pad_side_id+1),
                        sample.chip_geometry.sample_vertical_size-pad_offset), 
                        np.pi/2, tl_core, tl_gap, tl_ground, 
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry, 
                       **elements.default_pad_geometry())
    pads_top.append(pad)
    sample.add(pad)
    
p1 = pads_left[0]
p2 = pads_right[0]

In [4]:
g5, g6, res, open_part_of_res = draw_single_resonator(sample,elements,
                            coupler_start, central_line_y, coupler_length,
                             resonator_core,resonator_gap,resonator_ground, 
                             tl_core, tl_gap, tl_ground, grounding_width=10,
                             closed_end_meander_length=3000,length_left=200,length_right=400,
                             open_end_legth=800, min_bridge_spacing=150,
                             port_orientation='right', direction_orientation='down')

cpw segments [{'type': 'endpoint', 'endpoint': array([2261., 2299.]), 'length': 0}, {'type': 'endpoint', 'endpoint': array([2261., 1499.]), 'length': 800.0, 'startpoint': array([2261., 2299.])}]


In [5]:
connector = sample.connect_cpw(o1=p1, o2=g5, port1='port', port2='narrow', name='---',
                    points=[], bridges = True, pads_geometry=[4,10],
                    bridge_geometry=[56, 7], distance_between_pads=54, min_spacing=150)

cpw segments [{'type': 'endpoint', 'endpoint': array([ 800., 2350.]), 'length': 0}, {'type': 'segment', 'startpoint': array([ 800., 2350.]), 'endpoint': array([1764.6653303 , 2368.97524429]), 'length': 964.8519365085308}, {'type': 'turn', 'turn': -0.01966775104340696, 'startpoint': array([1764.6653303 , 2368.97524429]), 'endpoint': array([1767.18264014, 2369.        ]), 'length': -2.517472133556091, 'instead_point': array([1765.92386349, 2369.        ])}, {'type': 'endpoint', 'endpoint': array([1768., 2369.]), 'length': 0.8173598620257962, 'startpoint': array([1767.18264014, 2369.        ])}]
[CPW "---s0", l=152.475, CPW "---s1", l=152.475, CPW "---s2", l=152.475, CPW "---s3", l=152.475, CPW "---s4", l=152.475, CPW "---s5", l=155.810]


In [6]:
connector = sample.connect_cpw(o1=g6, o2=p2, port1='narrow', port2='port', name='---',
                    points=[], bridges = True, pads_geometry=[4,10],
                    bridge_geometry=[56, 7], distance_between_pads=54, min_spacing=150)

cpw segments [{'type': 'endpoint', 'endpoint': array([2282., 2369.]), 'length': 0}, {'type': 'endpoint', 'endpoint': array([3900., 2350.]), 'length': 1618.1115536328143, 'startpoint': array([2282., 2369.])}]
[CPW "---s0", l=152.811, CPW "---s1", l=152.811, CPW "---s2", l=152.811, CPW "---s3", l=152.811, CPW "---s4", l=152.811, CPW "---s5", l=152.811, CPW "---s6", l=152.811, CPW "---s7", l=152.811, CPW "---s8", l=152.811, CPW "---s9", l=152.811]


In [ ]:
sample.draw_design()
sample.watch()

C:\ProgramData\Anaconda3\envs\py38\lib\site-packages\gdspy\path.py:827: UserWarning: [GDSPY] Possible inconsistency found in `FlexPath` due to sharp corner.
  warnings.warn(


In [8]:
sample.write_to_gds()

Gds file has been writen here:  D:\Documents-cloud\qubits\simulation\QCreator\example_notebooks\1Q_test.gds


In [11]:
sys, connections, elements = sample.get_tls()

In [12]:
sample.get_topology()

[[Short , [0]],
 [Port pad-left-0, [81]],
 [Port pad-right-0, [92]],
 [Port pad-bottom-0, [93]],
 [Port pad-top-0, [94]],
 [Port pad-bottom-1, [95]],
 [Port pad-top-1, [96]],
 [Port pad-bottom-2, [97]],
 [Port pad-top-2, [98]],
 [TL TL-resonator coupler (n=3), [1, 2, 3, 6, 7, 8]],
 [TL closed end resonator fanout_line_0 (n=2), [99, 100, 4, 5]],
 [TL closed end resonator fanout_line_1 (n=1), [101, 73]],
 [TL closed end resonator fanout_coupled_line_2 (n=3),
  [1, 2, 3, 99, 100, 101]],
 [R cl10, [5, 0]],
 [TL cl1 (n=1), [4, 82]],
 [TL open end resonator fanout_line_0 (n=1), [102, 75]],
 [TL open end resonator fanout_line_1 (n=2), [103, 104, 9, 10]],
 [TL open end resonator fanout_coupled_line_2 (n=3), [8, 7, 6, 102, 103, 104]],
 [R cl20, [9, 0]],
 [TL cl2 (n=1), [10, 91]],
 [TL closed ends0 (n=1), [73, 11]],
 [TL closed endb0_line1 (n=1), [11, 105]],
 [TL closed endb0_line2 (n=1), [105, 12]],
 [C , [105, 0]],
 [TL closed ends1 (n=1), [12, 13]],
 [TL closed endb1_line1 (n=1), [13, 106]],


In [ ]:
omega, kappa, modes = sys.get_modes()

In [ ]:
omega / (2* np.pi)

In [ ]:
1.930694